In [27]:
# this cell contains dependencies for this notebook

import pandas as pd
import matplotlib.pyplot as plt
import numpy as nm
import math
import pylab
import matplotlib
import os
from sympy.solvers import solve
from sympy import Symbol


In [28]:
tp=0. # time of ponding, set to 0 by default, but the user 
      # can input a different constant value, and solutions 
      # will remain valid
m = 2.

# the following functions are the analytical equations 
# used by the next block to solve 

# all symbols are explicated in the table in the README document

# Domain 1 equations:
def h1(t,r,tr): # equation for average depth
    return (r-ks)*(t-tp)-A0*(t**0.5-tp**0.5)
def x1(t,x0,r,tr):
    return 2.*alpha*(0.5*(r-ks)*(t**2.-tp**2.)-2./3.*A0*(t**(3./2.)-tp**(3./2.))) + x0

# Domain 2 equations
def h2(t,t0,r,tr): # equation for average depth
    return (r-ks)*(t-t0)-A0*(t**0.5-t0**0.5)
def x2(t,t0,r,tr):
    return 2.*alpha*(-A0*(2./3.*t**(3./2.)+1./3.*t0**(3./2.)-t*t0**0.5)+(r-ks)*(0.5*t**2.+0.5*t0**2.-t*t0))


# Domain 3 equations:   
def h3(t,hstar,r,tr): # equation for average depth
    return -A0*(t**0.5-tr**0.5)-ks*(t-tr)+hstar
def x3(t,xstar,hstar,r,tr):
    return 2.*alpha*(-A0*(2./3.*t**(3./2.)+1./3.*tr**(3./2.)-t*tr**0.5)-ks*(0.5*t**2.+0.5*tr**2.-t*tr)+hstar*(t-tr))+xstar

In [63]:
# User-defined parameters
alpha = 0.05**(0.5)/0.05/1000.**(1./3.)              # roughness parameter S^0.5/n [n]=s/mm^1/3
outlet = 1.                # outlet width (mm)
xend = 100000.               # location of stream outlet (mm)
xtop = 0.                # location of divide (mm)
name = "Desktop/variable_c_data/rational_longhill_extralong/rational_info_"  # path to where output data should go
res = 100               # resolution of data for each case

In [64]:
# dictionary that holds data for each run before printing to csv file
rational_info = {'tc':[],'D':[],'C':[],'A':[],'I':[],'Q':[],'Qrat':[],'Creal':[]}

In [59]:
def make_rational(ks,A0,b,K,name,tend):
    r = K*tend**(-b)    # rainfall rate given by IFD function
    time = Symbol('time',positive=True)
    tp = (A0/(r-ks))**2.
    tc = solve(2.*alpha*(0.5*(r-ks)*(time**2.-tp**2.)-2./3.*A0*(time**(3./2.)-tp**(3./2.))) -xend,time) # solve for time of concentration

    try:
        tc = float(tc[0])
    except:
        tc = 0

    # dictionaries holding solution information
    touts_tot=[] # a list of time values at which depth is collected for each value of a
    houts_tot=[] # a list of depths corresponding to the time values collected for each value of a

    
    for tr in [3.*tc,tend]:
        #print r
        #solution begins below
        xstarvals = []                           # list of characteristic locations at tr
        hstarvals = []                           # list of depths associated with characteristic locations at tr
        houts1 = []                              # list of depths at outlet from domain 1
        touts1 = []                              # list of times associated with outlet depths from domain 1
        houts2 = []                              # list of depths at outlet from domain 2
        touts2 = []                              # list of times associated with outlet depths from domain 2
        houts3 = []                              # list of depths at outlet from domain 3
        touts3 = []                              # list of times associated with outlet depths from domain 3
        xstarplot = []                           # list of xstar vals used to plot characteristics starting at t=tr



        # Solution for Domain 1

        x0vals = nm.linspace(xtop,xend,100)             # initiate a list of 100 x0 values to start characteristics
        lastcross = -1
        for val in x0vals:                              # for loop solves a characteristic for each x0
            tvals= nm.linspace(tp,tr,100)           # initiates 100 tvals to solve the characteristic on
            xvals= x1(tvals,val,r,tr)                         # finds x at each t value using domain 1 equaion
            hvals= h1(tvals,r,tr)                        # finds depth using domain 1 equation at each tval

            if x1(tr,val,r,tr)>=xend:                 # if statement tests whether the characteristic crosses the outlet before tr
                cross = 0                           # find when the characteristic reaches the outlet
                for i in range(len(xvals)):
                    if xvals[i]>xend and cross==0:
                        cross = i-1
                houts1 = houts1+[hvals[cross]]               # if so, then the depth and time of crossing are kept for the hydrograph
                touts1 = touts1+[tvals[cross]]
            else:
                lastcross = val
                xstarvals=xstarvals+[x1(tr,val,r,tr)]            # if not, then the location xstar and depth hstar for this characteristic
                hstarvals=hstarvals+[h1(tr,r,tr)]                # are saved to send to domain 3

        touts1.reverse()                                     # reverses the elements of touts1 since they are backwards
        houts1.reverse()                                     # reverses the elements of houts1 to stay consistent with touts1

        # Solution for Domain 2

        t0vals = nm.linspace(tp,tr,100)                       # initiates a set of t0vals to start characteristics in domain 2
        for val in t0vals:                                   # for loop solves a characteristic for each t0
            tvals = nm.linspace(val,tr,10000)              # initiates a set of xvals to solve characteristic on
            hvals = h2(tvals,val,r,tr)                       # finds depth along characteristic using domain 2 equation
            xvals = x2(tvals,val,r,tr)                          # solves for x(t) characteristic using domain 2 equation

            stop=-1
            for i in range(len(hvals)):
                if hvals[i]<0:
                    i=stop
            if stop>=0:
                hvals = hvals[:stop]
                tvals = tvals[:stop]
                xvals = xvals[:stop]
            #print hvals
            if max(xvals)>=xend:                 # if statement tests whether the characteristic crosses the outlet before tr
                cross = 0                           # find when the characteristic reaches the outlet
                for i in range(len(xvals)):
                    if xvals[i]>xend and cross==0:
                        cross = i
                houts2 = houts2+[hvals[cross]]               # if so, then the depth and time of crossing are kept for the hydrograph
                touts2 = touts2+[tvals[cross]]
            elif x2(tr-val,val,r,tr)>0:
                xstarvals=xstarvals+[x2(tr,val,r,tr)]            # if not, then the location xstar and depth hstar for this characteristic
                hstarvals=hstarvals+[h2(tr,val,r,tr)]                # are saved to send to domain 3



        xzwvals=[]          # list of locations at which characteristic dries up (used only when f!=0)
        tzwvals=[]          # list of times at which characteristic dries up (used only when f!=0)

        for i in range (len(xstarvals)):                       # for loop solves characteristic for each xstarval
            tvals = nm.linspace(tr,4.*tr,100)        # initiates set of xvals to solve the characteristic on
            hvals = h3(tvals,hstarvals[i],r,tr)        # solves for depths along characteristic using domain 3 equation
            xvals = x3(tvals,xstarvals[i],hstarvals[i],r,tr)        # solves for t(x) characteristic using domain 3 equation



            for i in range(len(hvals)):
                idx = -1
                if hvals[i]<0 and idx==0:
                        idx = i
            if idx!=-1:
                tzwvals = tzwvals + [tvals[idx]]
                xzwvals = xzwvals + [xvals[idx]]
                hvals = hvals[:idx]
                tvals = tvals[:idx]
                xvals = xvals[:idx]


            if max(xvals)>=xend:                                         # not all characteristics leave landscape. If they leave, then
                out = 0.
                for i in range(len(xvals)):
                    if xvals[i]>=xend and out==0:
                        out = i
                houts3 = houts3+[hvals[out]]     # tracks depth
                touts3 = touts3+[tvals[out]]     # and time when they leave



        houts=houts1+houts2+houts3    # aggregate all depth data at outlet into one list
        touts=touts1+touts2+touts3    # aggregate all time data at outlet into one list


        houts_tot=houts         # save depth data to the dictionary houts_tot
        touts_tot=touts         # save time data to the dictionary touts_tot

        if tr==tend:
            
            qsum = 0
            for i in range(len(touts_tot)-1):
                qsum = qsum + (0.5*alpha*outlet*(houts_tot[i+1]**m+houts_tot[i]**m))*(touts_tot[i+1]-touts_tot[i])

            totalrain = r*tr*outlet*xend    
            Creal = qsum/totalrain
 
            A = 0
            if houts2!=[]:
                A = outlet*xend
            else:
                A = (xend-lastcross)*outlet




            rational_info['A'] = rational_info['A'] + [A]
            rational_info['I'] = rational_info['I'] + [r]
            try:
                rational_info['Q'] = rational_info['Q'] + [max(houts1)**m*outlet*alpha]
            except:
                rational_info['Q'] = rational_info['Q'] +[0]
            rational_info['Qrat'] = rational_info['Qrat']+ [C*A*r]
            rational_info['D'] = rational_info['D']+[tr]
            rational_info['tc'] = rational_info['tc']+[tc]
            rational_info['Creal'] = rational_info['Creal']+[Creal]


            A = rational_info['A'][-1]
            I = rational_info['I'][-1]
            Q = rational_info['Q'][-1]
            Qrat = rational_info['Qrat'][-1]
            D = rational_info['D'][-1]
            tc = rational_info['tc'][-1]
            C = rational_info['C'][-1]
            f.write(str(D)+ ', '+ str(tc)+ ', '+str(A)+', '+str(I)+', '+str(Q)+', '+str(Qrat)+', '+str(C)+' , '+str(Creal)+'\n')
            
        else:
            qsum = 0
            for i in range(len(touts_tot)-1):
                qsum = qsum + (0.5*alpha*outlet*(houts_tot[i+1]**m+houts_tot[i]**m))*(touts_tot[i+1]-touts_tot[i])

            totalrain = r*tr*outlet*xend    
            C = qsum/totalrain
            rational_info['C'] = rational_info['C']+[C]
    

In [71]:
# this block runs the code and outputs to csv files at the user-defined location

# parameters that do not change between runs
ksvals = [5.*10**(-2.),3.*10**(-3.),1.*10**(-4.),0.] # sand, loam, clay (mm/s)
bvals = nm.linspace(0.35,0.65,5)  # I = K*D^(-b), this range is "reasonable"
Kvals = [0.1,0.5,1.,2.5,5.,7.5,10.] # range of values for K (mm/s)
tttvals={0.1:12000,0.5:12000,1.:12000,2.5:10000,5.:4000,7.5:3500,10.:2500} # simulation maximum run times for each K value

# The following runs the optimization and tracks Q, C, A, I for each storm duration under the
# range of ks, A0, and K values 
# writes data to csv files located at the user-defined location

for ks in ksvals:
    if ks==0.: # when ks=0, A0=0 always
        A0vals = [0.] # mm/s^1/2
    if ks == ksvals[0]: #sand
        A0vals = [2.,1.,0.1,0.01,0.]
    elif ks == ksvals[1]: #loam
        A0vals = [1.,0.1,0.01,0.001,0.0] # mm/s^1/2
    if ks == ksvals[2]: #clay
        A0vals = [0.1,0.01,0.001,0.0001,0.] # mm/s^1/2
    for K in Kvals:
        for A0 in A0vals:
            trvals = nm.linspace(1,tttvals[K],res)
            for b in bvals:
                rational_info = {'tc':[],'D':[],'C':[],'A':[],'I':[],'Q':[],'Qrat':[],'Creal':[]}
                f = open(name+str(ks)+"_"+str(K)+"_"+str(A0)+"_"+str(b)+".csv","w")
                f.write('D,tc,A,I,Q,Qrat,C,Creal \n')
                for tend in trvals:
                    make_rational(ks,A0,b,K,name,tend)
                f.close()
            print ('finished ks = ',ks,'A0 = ',A0)
        print ('finished ks = ',ks,'K = ',K)
    print ('finished ks = ',ks)
print ('done')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in double_scalars


finished ks =  0.05 A0 =  1.0
finished ks =  0.05 A0 =  0.0
finished ks =  0.05 K =  2.5
finished ks =  0.05 A0 =  1.0
finished ks =  0.05 A0 =  0.0
finished ks =  0.05 K =  7.5
finished ks =  0.05
finished ks =  0.003 A0 =  1.0
finished ks =  0.003 A0 =  0.1
finished ks =  0.003 A0 =  0.0
finished ks =  0.003 K =  2.5
finished ks =  0.003 A0 =  1.0
finished ks =  0.003 A0 =  0.1
finished ks =  0.003 A0 =  0.0
finished ks =  0.003 K =  7.5
finished ks =  0.003
finished ks =  0.0001 A0 =  0.1
finished ks =  0.0001 A0 =  0.01
finished ks =  0.0001 A0 =  0.0
finished ks =  0.0001 K =  2.5
finished ks =  0.0001 A0 =  0.1
finished ks =  0.0001 A0 =  0.01
finished ks =  0.0001 A0 =  0.0
finished ks =  0.0001 K =  7.5
finished ks =  0.0001
finished ks =  0.0 A0 =  0.0
finished ks =  0.0 K =  2.5
finished ks =  0.0 A0 =  0.0
finished ks =  0.0 K =  7.5
finished ks =  0.0
done
